In [1]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Ex_of_ML').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/29 14:24:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/01/29 14:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [10]:
#Read The Data Set
traning=spark.read.csv('/Users/pragatigupta/Documents/OneDrive - montclair.edu/MSU/Interview/Big DATA/python Spark/test6.csv',header=True,inferSchema=True)

In [11]:
traning.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|  RAM| 31|        10| 30000|
| Sham| 30|         8| 25000|
|Reena| 29|         4| 20000|
|Teena| 24|         3| 20000|
| Hema| 21|         1| 15000|
|kamal| 23|         2| 18000|
+-----+---+----------+------+



In [12]:
traning.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [14]:
traning.columns

['Name', 'Age', 'Experience', 'Salary']

22/01/29 14:49:16 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1060251 ms exceeds timeout 120000 ms
22/01/29 14:49:16 WARN SparkContext: Killing executors is not supported by current scheduler.


In [27]:
#[Age,Experience]---->new feature ---->independent feature   via VectorAssembler
# Not for string,
# for Categorical dataset : first convert then into numeriacal values

# VectorAssembler

In [28]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Age','Experience'],outputCol='independent Features')

In [29]:

output=featureassembler.transform(traning)

In [30]:
output.show()

+-----+---+----------+------+--------------------+
| Name|Age|Experience|Salary|independent Features|
+-----+---+----------+------+--------------------+
|  RAM| 31|        10| 30000|         [31.0,10.0]|
| Sham| 30|         8| 25000|          [30.0,8.0]|
|Reena| 29|         4| 20000|          [29.0,4.0]|
|Teena| 24|         3| 20000|          [24.0,3.0]|
| Hema| 21|         1| 15000|          [21.0,1.0]|
|kamal| 23|         2| 18000|          [23.0,2.0]|
+-----+---+----------+------+--------------------+



In [31]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'independent Features']

In [33]:
finalized_data=output.select('independent Features','Salary')

In [35]:
finalized_data.show()
#Here independent Features: independent Features and Dependent Feature: Salary 

+--------------------+------+
|independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [ ]:
#train-test split finalized_data.randomSplit([]) 
#featureCol: input
#labelCol : output 

# LinearRegression

In [41]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

22/01/29 15:33:10 WARN Instrumentation: [554598a0] regParam is zero, which might cause numerical instability and overfitting.
22/01/29 15:33:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/01/29 15:33:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/01/29 15:33:12 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [42]:
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [43]:
regressor.intercept

16470.03994673731

# Prediction

In [45]:
pred_results=regressor.evaluate(test_data)

In [46]:
pred_results.predictions.show()

/Users/pragatigupta/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+-----------------+
|independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



# meanAbsoluteError

In [50]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)